In [1]:
%pip install qdrant-client sentence-transformers transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install langchain==0.2.0

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install --upgrade langchain langchain-experimental



  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.0
    Uninstalling langchain-0.2.0:
      Successfully uninstalled langchain-0.2.0
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install groq

Note: you may need to restart the kernel to use updated packages.


In [1]:
# ---------------------------------------------------------
# 1. IMPORTS
# ---------------------------------------------------------
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ---------------------------------------------------------
# 2. LOAD YOUR DOCUMENT
# ---------------------------------------------------------
print("🔵 Loading documents from demo.txt...")

with open("demo.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print(f"📄 Loaded {len(raw_text)} characters")

# ---------------------------------------------------------
# 3. SEMANTIC CHUNKING  (BEST QUALITY CHUNKING)
# ---------------------------------------------------------
print("🧠 Running Semantic Chunker...")

semantic_embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

semantic_chunker = SemanticChunker(
    embeddings=semantic_embedder,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95
)

semantic_chunks = semantic_chunker.split_text(raw_text)

print("🔶 Semantic chunking done.")
print(f"Chunks: {len(semantic_chunks)}")

# ---------------------------------------------------------
# 4. RECURSIVE SLIDING WINDOW CHUNKING
# ---------------------------------------------------------
print("🧩 Applying recursive sliding window...")

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=80,
    separators=["\n\n", "\n", ".", "!", "?"]
)

final_chunks = []
for chunk in semantic_chunks:
    sub_chunks = recursive_splitter.split_text(chunk)
    final_chunks.extend(sub_chunks)

print(f"🟢 Total final chunks after semantic + recursive: {len(final_chunks)}")

# ---------------------------------------------------------
# 5. EMBEDDINGS USING BGE
# ---------------------------------------------------------
print("🧬 Loading BGE model...")

from sentence_transformers import SentenceTransformer
bge = SentenceTransformer("BAAI/bge-base-en")
bge.max_seq_length = 512

def embed(batch):
    return bge.encode(batch, normalize_embeddings=True).tolist()

# ---------------------------------------------------------
# 6. QDRANT SETUP
# ---------------------------------------------------------
print("🗄️ Initializing Qdrant...")

qdrant = QdrantClient(path="qdrant_db")

collection_name = "demo_collection"

# Create / recreate collection
qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=768,     # BGE vector dimension
        distance=Distance.COSINE
    )
)

print("📌 Collection created.")

# ---------------------------------------------------------
# 7. STORE EMBEDDINGS IN QDRANT
# ---------------------------------------------------------
print("📥 Storing embeddings in Qdrant...")

points = []
for idx, chunk in enumerate(final_chunks):
    vector = embed([chunk])[0]

    points.append(
        PointStruct(
            id=idx,
            vector=vector,
            payload={"text": chunk}
        )
    )

qdrant.upsert(collection_name=collection_name, points=points)

print(f"✅ Stored {len(points)} embeddings.")
print("🎉 Ingestion complete!")


c:\Users\sashi\anaconda3\envs\qaagent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔵 Loading documents from demo.txt...
📄 Loaded 55878 characters
🧠 Running Semantic Chunker...


C:\Users\sashi\AppData\Local\Temp\ipykernel_12780\2343795345.py:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  semantic_embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


🔶 Semantic chunking done.
Chunks: 7
🧩 Applying recursive sliding window...
🟢 Total final chunks after semantic + recursive: 222
🧬 Loading BGE model...
🗄️ Initializing Qdrant...
📌 Collection created.
📥 Storing embeddings in Qdrant...


C:\Users\sashi\AppData\Local\Temp\ipykernel_12780\2343795345.py:79: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


✅ Stored 222 embeddings.
🎉 Ingestion complete!


In [37]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer, CrossEncoder

# 1. Qdrant connection
#qdrant = QdrantClient(path="qdrant_db")
#collection_name = "demo_collection"

# 2. BGE embedding model
#embed_model = SentenceTransformer("BAAI/bge-base-en")
#embed_model.max_seq_length = 512

# 3. BGE reranker (cross-encoder)
reranker = CrossEncoder("BAAI/bge-reranker-base")



In [38]:
# Embed query and vector search using qdrant

def embed_query(text):
    return bge.encode(text, normalize_embeddings=True).tolist()


def vector_search(query, top_k=10):
    vector = embed_query(query)

    results = qdrant.query_points(
        collection_name=collection_name,
        query=vector,
        limit=top_k
    )

    docs = []
    for hit in results.points:     # 👈 Correct source of hits
        docs.append({
            "text": hit.payload.get("text", ""),
            "score_before": hit.score
        })

    return docs


In [43]:
# Rerank using BGE reranker

from sentence_transformers import CrossEncoder

reranker = CrossEncoder("BAAI/bge-reranker-base")
def rerank_results(query, docs, top_k=5):
    pairs = [(query, d["text"]) for d in docs]       # (query, passage) pairs
    scores = reranker.predict(pairs)                 # CrossEncoder scoring

    ranked = []
    for d, s in zip(docs, scores):
        ranked.append({
            "text": d["text"],
            "score_after": float(s)
        })

    # Highest score = most relevant
    ranked = sorted(ranked, key=lambda x: x["score_after"], reverse=True)

    return ranked[:top_k]






In [44]:
# Calculate Hit@K, MRR, Recall@K

def hit_at_k(retrieved_docs, ground_truth, k):
    texts = [d["text"] for d in retrieved_docs[:k]]
    return 1 if any(ground_truth in t for t in texts) else 0


def mrr(retrieved_docs, ground_truth):
    for idx, d in enumerate(retrieved_docs):
        if ground_truth in d["text"]:
            return 1 / (idx + 1)
    return 0

def recall_at_k(retrieved_docs, ground_truth, k):
    texts = [d["text"] for d in retrieved_docs[:k]]
    return 1 if any(ground_truth in t for t in texts) else 0


In [57]:
def evaluate(query, ground_truth, top_k=10, rerank_k=10):

    # BEFORE
    docs_before = vector_search(query, top_k=top_k)
    hit_before = hit_at_k(docs_before, ground_truth, top_k)
    mrr_before = mrr(docs_before, ground_truth)
    recall_before = recall_at_k(docs_before, ground_truth, top_k)

    # AFTER
    docs_after = rerank_results(query, docs_before, top_k=rerank_k)
    hit_after = hit_at_k(docs_after, ground_truth, rerank_k)
    mrr_after = mrr(docs_after, ground_truth)
    recall_after = recall_at_k(docs_after, ground_truth, rerank_k)

    return {
        "before": {"docs": docs_before, "hit@k": hit_before, "mrr": mrr_before, "recall@k": recall_before},
        "after": {"docs": docs_after, "hit@k": hit_after, "mrr": mrr_after, "recall@k": recall_after},
    }





In [59]:
# Printing:

query = "Who was identified as the perpetrator of the 2025 Delhi car explosion?"
ground_truth = "Umar Mohammed"

results = evaluate(query, ground_truth)

print("\n==== BEFORE RERANKING ====")
print("Hit@K:", results["before"]["hit@k"])
print("MRR:", results["before"]["mrr"])
print("Recall@K:", results["before"]["recall@k"])

print("\n==== AFTER RERANKING ====")
print("Hit@K:", results["after"]["hit@k"])
print("MRR:", results["after"]["mrr"])
print("Recall@K:", results["after"]["recall@k"])

print("\n🔍 Top Passages (Before):")
for d in results["before"]["docs"]:
    print(d["score_before"], "->", d["text"][:120], "...")

print("\n🔍 Top Passages (After):")
for d in results["after"]["docs"]:
    print(d["score_after"], "->", d["text"][:120], "...")



==== BEFORE RERANKING ====
Hit@K: 1
MRR: 0.3333333333333333
Recall@K: 1

==== AFTER RERANKING ====
Hit@K: 1
MRR: 1.0
Recall@K: 1

🔍 Top Passages (Before):
0.888073228094696 -> On 10 November 2025, a car exploded near the Red Fort in Delhi, India, killing at least 13 people and injuring more than ...
0.8762185539534353 -> At 6:52 pm IST on 10 November 2025, a car exploded near the Red Fort in Delhi.[7][8] The car, which was identified as a  ...
0.8527885230923629 -> .[18][29][30] As per media reports, the investigators allegedly traced the ownership of the car to Umar Mohammed, a Pulw ...
0.8501196396374484 -> Weapons	Car loaded with ammonium nitrate fuel oil and other explosives
Deaths	13 (including one perpetrator)
Injured	20+ ...
0.8474811010956231 -> .[25][26] As per police reports, the blast is suspected to have been caused by ammonium nitrate fuel oil, which triggere ...
0.8452728483028067 -> .[11][12] As per Satish Golcha, the commissioner of the Delhi Police, after leaving the 

In [47]:
def build_prompt(question, passages):
    context = "\n\n".join([f"- {p['text']}" for p in passages])

    prompt = f"""
You are an expert assistant. Use ONLY the context given in the PDF to answer the question.
If the answer is not in the context, say "I do not know."

### Context:
{context}

### Question:
{question}

### Answer:
"""

    return prompt


In [51]:
# Generate final answer iusing LLM

from groq import Groq
import os 
from dotenv import load_dotenv



load_dotenv()

groq_client = Groq(api_key = os.getenv('GROQ_API_KEY'))

def generate_answer(prompt):
    response = groq_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return response.choices[0].message.content


In [52]:
# Aygumentation pipeline

def answer_question(question):
    # 1. Retrieve
    retrieved_docs = vector_search(question, top_k=10)

    # 2. Rerank (optional but recommended)
    reranked_docs = rerank_results(question, retrieved_docs, top_k=5)

    # 3. Build prompt
    prompt = build_prompt(question, reranked_docs)

    # 4. LLM answer
    answer = generate_answer(prompt)

    return answer, reranked_docs


In [53]:
question = "What does the document say about Phalgam attack?"
answer, contexts = answer_question(question)

print("🔵 Answer:")
print(answer)

print("\n🔶 Contexts used:")
for c in contexts:
    print("—", c["text"][:120], "...")


🔵 Answer:
The document mentions the following about the Pahalgam attack:

- It occurred in 2025 as part of the Insurgency in Jammu and Kashmir.
- The attack was condemned by the Financial Action Task Force, which stated that such attacks cannot occur "without money and the means to move funds between terrorist supporters".
- The terrorist group TRF initially took responsibility for the attack.
- Pakistan's deputy prime minister and foreign minister, Ishaq Dar, referred to the perpetrators as "freedom fighters" and questioned the evidence linking TRF to the attack.
- The hashtags #IndianFalseFlag and #PahalgamDramaExposed trended briefly on the social media platform X, with a majority of posts originating from Pakistan.

🔶 Contexts used:
— .[369][370] Post Pahalgam attack the hashtags #IndianFalseFlag and #PahalgamDramaExposed trended briefly on the social m ...
— If you agree, consider giving $25 or even just $2.75. Give $2.75
 Give a different amount
Maybe later 
I already donated ...